## CRF model

Modelo CRF para ato de aposentadoria (com dados rotulados manualmente)

In [61]:
# !pip install sklearn_crfsuite
# !pip install nltk
# !pip install pandas
import pandas as pd
import sklearn_crfsuite
import nltk
from nltk.tokenize import word_tokenize

In [62]:
tokens = '/home/thais/Documentos/Knedle/experiments/members/jose_reinaldo/TCDF_aposentadoria/raw_tokenized_atos.parquet'
texts = '/home/thais/Documentos/Knedle/experiments/members/jose_reinaldo/TCDF_aposentadoria/labelbox_texts_atos.parquet'
# tokens tratados
texts = "/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/result/labelbox_atos.parquet"
tokens = "/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/result/tokenized_texts.parquet"

df1 = pd.read_parquet(tokens)
df2 = pd.read_parquet(texts)

df2.groupby('ato').count()
# df2["ato"].value_counts()

# result = pd.merge(df1, df2, on=["arquivo_rast"])
# result.to_csv('result.csv')
# result.head()

,arquivo_rast,text,dodf,treated_text
ato,,,,
aviso_de_aditamento_contratual,463,463,463,463
aviso_de_licitacao,235,235,235,235
aviso_de_revogacao_anulacao_de_licitacao,16,16,16,16
aviso_de_suspensao_de_licitacao,12,12,12,12
extrato_de_contrato_ou_convenio,442,442,442,442


In [63]:
df= pd.read_parquet('iob_aviso_licitacao.parquet')
result = pd.DataFrame(df)


Load csv with text and labels

In [64]:
from nltk.tokenize import RegexpTokenizer

TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize

x = []
y = []
for row in range(len(result)):
    if pd.notna(result['treated_text'][row]):
        # text_iob = result['IOB'][row].decode("utf-8").replace("', '", '').replace("[' ", '').replace("']", '')
        x.append(TOKENIZER(result['treated_text'][row]))
        y.append(result['IOB'][row].split())
len(x), len(y)

(234, 234)

Create dictionary feature for each word in each sequence in x

In [65]:
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            # print(sentence[i])
            word_feat = {
                # Palavra atual
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                # Uma palavra antes
                'word_before': '' if i == 0 else sentence[i-1].lower(),
                'word_before_isdigit': '' if i == 0 else sentence[i-1].isdigit(),
                'word_before_isupper': '' if i == 0 else sentence[i-1].isupper(),
                'word_before_istitle': '' if i == 0 else sentence[i-1].istitle(),

                # Uma palavra depois
                'word_after': '' if i+1 >= len(sentence) else sentence[i+1].lower(),
                'word_after_isdigit': '' if i+1 >= len(sentence) else sentence[i+1].isdigit(),
                'word_after_isupper': '' if i+1 >= len(sentence) else sentence[i+1].isupper(),
                'word_after_istitle': '' if i+1 >= len(sentence) else sentence[i+1].istitle(),

                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x)):
    x[i] = get_features(x[i])

In [66]:
x[0]

[{'word': 'aviso',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': '',
  'word_before_isdigit': '',
  'word_before_isupper': '',
  'word_before_istitle': '',
  'word_after': 'de',
  'word_after_isdigit': False,
  'word_after_isupper': True,
  'word_after_istitle': False,
  'BOS': True,
  'EOS': False},
 {'word': 'de',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'aviso',
  'word_before_isdigit': False,
  'word_before_isupper': True,
  'word_before_istitle': False,
  'word_after': 'licitação',
  'word_after_isdigit': False,
  'word_after_isupper': True,
  'word_after_istitle': False,
  'BOS': False,
  'EOS': False},
 {'word': 'licitação',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'de',
  'word_before_isdigit': False,
  'word_before_isupper': True,
  'word_before_istitle': False,
  'word_after': 'pregão',
  'word_after_isdigit': False,
  'word_after_isupper': True,
 

Separate train and test splits (in order)

In [67]:
import math
split_idx = math.floor(0.8*len(x))
x_train = x[0:split_idx]
y_train = y[0:split_idx]
x_test = x[split_idx:]
y_test = y[split_idx:]
len(x_train),  len(x_test), len(y_train)


(187, 47, 187)

In [68]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm = 'lbfgs',
    c1=0.17,
    c2=0.17,
    max_iterations=50,
    all_possible_transitions=True
)

crf.fit(x_train, y_train)

/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/venv/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.17, c2=0.17,
    keep_tempfiles=None, max_iterations=50)

In [69]:
classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.8118944528193618

In [70]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=classes, digits=3
))

/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['B-MODALIDADE_LICITACAO', 'I-MODALIDADE_LICITACAO', 'B-NUM_LICITACAO', 'I-NUM_LICITACAO', 'B-PROCESSO', 'I-PROCESSO', 'B-OBJ_LICITACAO', 'I-OBJ_LICITACAO', 'B-VALOR_ESTIMADO', 'I-VALOR_ESTIMADO', 'B-CODIGO_SISTEMA_COMPRAS', 'B-SISTEMA_COMPRAS', 'I-SISTEMA_COMPRAS', 'B-DATA_ABERTURA', 'I-DATA_ABERTURA', 'B-TIPO_OBJ', 'I-TIPO_OBJ', 'B-ORGAO_LICITANTE', 'I-ORGAO_LICITANTE', 'I-CODIGO_SISTEMA_COMPRAS'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with 

                          precision    recall  f1-score   support

  B-MODALIDADE_LICITACAO      0.976     0.854     0.911        48
  I-MODALIDADE_LICITACAO      1.000     0.765     0.867        51
         B-NUM_LICITACAO      0.942     0.875     0.907        56
         I-NUM_LICITACAO      0.942     0.961     0.951        51
              B-PROCESSO      0.821     0.548     0.657        42
              I-PROCESSO      0.842     0.577     0.684       111
         B-OBJ_LICITACAO      0.949     0.822     0.881        45
         I-OBJ_LICITACAO      0.817     0.866     0.841      1442
        B-VALOR_ESTIMADO      0.857     0.353     0.500        17
        I-VALOR_ESTIMADO      1.000     0.459     0.630        37
B-CODIGO_SISTEMA_COMPRAS      0.200     0.167     0.182         6
       B-SISTEMA_COMPRAS      0.811     0.811     0.811        37
       I-SISTEMA_COMPRAS      0.785     0.906     0.841       117
         B-DATA_ABERTURA      0.857     0.692     0.766        52
         